
# CP5 – Modelo de Classificação com IA (Classificação de Câncer de Mama - UCI/Scikit-Learn)

**Aluno:** _Preencha com seu nome_  
**Disciplina:** Inteligência Artificial / Machine Learning  
**Entrega:** Projeto individual – Classificação supervisionada  

---

## Definição do Problema

### Objetivo
Classificar tumores de mama como **Malignos** ou **Benignos** a partir de medidas reais obtidas de imagens (features numéricas), caracterizando um **problema de classificação binária**.

### Justificativa
Câncer de mama é um problema de grande impacto em saúde pública. Modelos de classificação podem **auxiliar no apoio à decisão clínica**, priorizando exames e encaminhando pacientes para investigação adicional. O dataset escolhido é amplamente utilizado como benchmark em ML, com qualidade e documentação reconhecidas, o que facilita a **reprodutibilidade** e a **clareza metodológica**.



## Descrição do Dataset

- **Nome:** Breast Cancer Wisconsin (Diagnostic)  
- **Origem/Fonte:** UCI Machine Learning Repository (via scikit-learn)  
- **Link (referência):** https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+original  
- **Acesso:** via `sklearn.datasets.load_breast_cancer()`  

**Observação:** a versão disponibilizada pelo `scikit-learn` é a **Breast Cancer Wisconsin (Diagnostic)**.  
Contém **569 amostras** e **30 variáveis explicativas** (todas numéricas), mais a **variável-alvo** binária: 0 (maligno) e 1 (benigno).

As features incluem estatísticas (média, erro padrão, pior valor) de medidas como:
- raio, textura, perímetro, área, suavidade, compacidade, concavidade, número de pontos côncavos, simetria, dimensão fractal.

Este dataset **não contém dados sensíveis identificáveis** e é amplamente usado para ensino e pesquisa.


In [ ]:

# === Imports principais ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Para evitar warnings excessivos
import warnings
warnings.filterwarnings('ignore')


## Carregamento e Exploração Inicial (EDA)

In [ ]:

# Carrega dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

# Informações básicas
n_rows, n_cols = X.shape
classes, counts = np.unique(y, return_counts=True)

print(f"Linhas: {n_rows}, Colunas (features): {n_cols}")
print("Alvo (0=Maligno, 1=Benigno):", dict(zip(classes, counts)))

# Visualiza primeiras linhas
display(X.head())

# Verifica valores ausentes
missing = X.isnull().sum().sum()
print(f"Valores ausentes totais nas features: {missing}")


### Checagem simples de outliers (opcional)

In [ ]:

# Checagem simples via z-score (não vamos remover, apenas reportar)
from scipy.stats import zscore

z_scores = np.abs(zscore(X))
# Considera como "outlier" se z-score > 3 em qualquer feature (contagem por linha)
outlier_flags = (z_scores > 3).any(axis=1)
print(f"Amostras com algum z-score > 3: {outlier_flags.sum()} de {len(outlier_flags)} ({outlier_flags.mean()*100:.2f}%)")


## Divisão de Dados (train/test)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Tamanhos -> X_train:", X_train.shape, "| X_test:", X_test.shape)


## Modelagem


Modelos testados (mínimo 3 exigidos pela atividade):

- **Regressão Logística** (com padronização)
- **LDA – Linear Discriminant Analysis** (com padronização)
- **RandomForest** (árvore de decisão em conjunto; não requer padronização)
- **KNN – k-vizinhos** (com padronização)

> Observação: usamos `Pipeline` para garantir que o `StandardScaler` seja aplicado **apenas** aos dados de treino dentro do fluxo de validação.


In [ ]:

models = {
    "LogisticRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=500, random_state=42))
    ]),
    "LDA": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LinearDiscriminantAnalysis())
    ]),
    "RandomForest": Pipeline([
        ("clf", RandomForestClassifier(n_estimators=300, random_state=42))
    ]),
    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=7))
    ]),
}

def evaluate_model(pipe, X_train, y_train, X_test, y_test):
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds)
    rec = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)
    report = classification_report(y_test, preds, target_names=["Maligno (0)", "Benigno (1)"])
    return {"acc": acc, "prec": prec, "rec": rec, "f1": f1, "cm": cm, "report": report, "model": pipe}

results = {}
for name, pipe in models.items():
    results[name] = evaluate_model(pipe, X_train, y_train, X_test, y_test)

# Tabela de métricas
metrics_df = pd.DataFrame({
    model: { 
        "Acurácia": res["acc"],
        "Precisão": res["prec"],
        "Recall": res["rec"],
        "F1-Score": res["f1"]
    }
    for model, res in results.items()
}).T.sort_values("F1-Score", ascending=False)

display(metrics_df.round(4))


### Matrizes de Confusão

In [ ]:

for name, res in results.items():
    cm = res["cm"]
    fig, ax = plt.subplots(figsize=(4, 3))
    im = ax.imshow(cm, interpolation='nearest')
    ax.set_title(f"Matriz de Confusão - {name}")
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    tick_marks = np.arange(2)
    ax.set_xticks(tick_marks)
    ax.set_yticks(tick_marks)
    ax.set_xticklabels(['Maligno (0)', 'Benigno (1)'])
    ax.set_yticklabels(['Maligno (0)', 'Benigno (1)'])
    ax.set_ylabel('Verdadeiro')
    ax.set_xlabel('Predito')

    # Anotações
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center")
    plt.show()


### Relatórios de Classificação (por modelo)

In [ ]:

for name, res in results.items():
    print(f"=== {name} ===")
    print(res["report"])


### Correlação entre features (opcional)

In [ ]:

corr = X.corr()
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(corr.values, aspect='auto')
ax.set_title("Mapa de Correlação (features)")
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
# Para não poluir, omitimos todos os rótulos (são 30). Em relatórios, recomenda-se salvar com alta resolução.
plt.show()



## Interpretação dos Resultados

- A tabela de métricas apresenta uma **comparação direta** entre os modelos.  
- O critério principal adotado aqui é o **F1-Score**, por equilibrar Precisão e Recall em um problema de saúde (reduz falsos negativos).  
- Em geral, modelos lineares como **LDA** e **Regressão Logística** costumam performar muito bem neste dataset por conta de sua separabilidade. **RandomForest** também tende a obter alto desempenho sem grande ajuste fino.

> **Modelo recomendado:** selecionar o **maior F1-Score** observado na sua execução (consulte a tabela exibida) e justificá-lo com base no equilíbrio entre Precisão e Recall.



## Conclusão

**Principais aprendizados:**
- A importância do **pré-processamento** (padronização) para modelos sensíveis à escala (LR, LDA, KNN).
- Avaliar com múltiplas métricas (**Acurácia, Precisão, Recall, F1**) dá uma visão mais robusta que uma métrica isolada.
- O dataset demonstra que **modelos lineares** podem ser muito competitivos quando as classes são bem separáveis.

**Melhorias futuras:**
- **Validação cruzada** e **ajuste de hiperparâmetros** (GridSearchCV/RandomizedSearchCV).
- Seleção/engenharia de features e análise de **importância de variáveis** (ex.: `coef_` da LR/LDA, `feature_importances_` da RandomForest).
- Análise de **curvas ROC/PR** com thresholds para balancear melhor Precisão vs. Recall conforme o objetivo clínico.



## Reprodutibilidade

- `random_state=42` onde aplicável (treino/teste e alguns modelos).  
- Ambiente recomendado: **Google Colab** ou **Jupyter Notebook** com Python 3.10+ e scikit-learn 1.3+.  
- Bibliotecas utilizadas: `pandas`, `numpy`, `scikit-learn`, `matplotlib`.



## Como Executar (Google Colab)

1. Faça upload deste arquivo `.ipynb` para o Google Drive e **abra com o Colab**.  
2. Execute as células **de cima para baixo**.  
3. Ao final, confira a **tabela de métricas** e justifique o **modelo final** com base nas métricas (especialmente F1-Score).

## Entregáveis Recomendados

- Repositório no GitHub contendo:
  - Este notebook (`.ipynb`)
  - Um `README.md` com descrição do problema, passos para execução e principais resultados
  - (Opcional) Imagens geradas (matrizes de confusão) caso deseje salvar figuras.



## Apêndice – Atendendo à Rubrica

- **Escolha do problema e dataset:** problema real e relevante (diagnóstico auxiliar), dataset público e confiável (UCI).  
- **Organização e clareza do código:** células separadas por etapa, comentários e prints claros.  
- **Pré-processamento:** verificação de ausentes, checagem simples de outliers, padronização em pipelines.  
- **Modelagem:** 4 modelos (≥3 exigidos) com `train_test_split`.  
- **Métricas:** acurácia, precisão, recall, F1 e matrizes de confusão.  
- **Conclusões e análise crítica:** interpretação comparativa e melhorias futuras.  
